In [1]:
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.tools as pytools
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
pytools.set_credentials_file(username='toonistic', api_key='S1ikiXlJEkgj6cuXPwLs')
import math
from glob import glob
import os
import sys
from tqdm import tqdm_notebook, tqdm
import re
from StringIO import StringIO
import numpy as np

## Distance to Target

In [2]:
def get_datadataframe(datafilename):
    rexp = r"(\([-0-9\.]+),( [-0-9\.]+),( [-0-9\.]+\),\([-0-9\.]+),( [-0-9\.]+),( [-0-9\.]+\),\([-0-9\.]+),( [-0-9\.]+),( [-0-9\.]+\),\([-0-9\.]+),( [-0-9\.]+),( [-0-9\.]+\))"
    rsubexp = r"\1|\2|\3|\4|\5|\6|\7|\8|\9"
    rpat = re.compile(rexp)
    processedcsv = ""
    with open(datafilename,'r') as dfile:
        headers = dfile.readline()
        processedcsv += ("date,time,"+headers)
        for line in dfile:
            processedcsv += (rpat.sub(rsubexp,line))
    return pd.read_csv(StringIO(processedcsv))

def process_datadataframe(filename, plot=True):
    datafilename = filename+".data.csv"
    targetdataframe = pd.read_csv(filename+".csv")
    datadf = get_datadataframe(datafilename)
    
    # Select only destination events
    datadf = datadf[(datadf.action == "POINTER DESTINATION") & (datadf.targetName != "button") & (datadf.targetName != "Plane")]
    # Remove duplicates
    datadf = datadf[datadf['targetName'] != datadf['targetName'].shift()]
    # Reset row labels
    datadf = datadf.reset_index()

    error = []
    for i in range(0, len(datadf)):
        cur = datadf.iloc[i]
        localtippos = cur['localTipPosition'][1:-1].split('|')
        localtargetpos = cur['localTargetPosition'][1:-1].split('|')
        error.append(math.sqrt(math.pow(float(localtippos[0])-float(localtargetpos[0]),2)+math.pow(float(localtippos[2])-float(localtargetpos[2]),2)))
    datadf['error'] = error
    
    timediff = []
    rounds = []
    trials = []
    sessions = []
    blind = []
    alerttone = []
    for i in tqdm_notebook(range(0,len(datadf))):
        cur = datadf.iloc[i]
        curtime = cur['time']
        targetrow = targetdataframe.iloc[0]
        for j in range(0,len(targetdataframe)):
            curtarget = targetdataframe.iloc[j]
    #         if curtarget['Trial'] == -1: continue
            if curtarget['TargetID'] == cur['targetName']:
                if targetrow['TargetID'] != cur['targetName'] or abs(curtarget['Millis']-curtime) < abs(targetrow['Millis']-curtime):
                    targetrow = curtarget
        if targetrow['TargetID'] != cur['targetName']:
            timediff.append(1000000)
        else:
            timediff.append(abs(targetrow['Millis']-curtime))
        rounds.append(targetrow['Round'])
        trials.append(targetrow['Trial'])
        sessions.append(targetrow['Session'])
        blind.append(targetrow['Blind'])
        alerttone.append(targetrow['AlertTone'])
    datadf['TimeDiff'] = timediff
    datadf['Round'] = rounds
    datadf['Trial'] = trials
    datadf['Session'] = sessions
    datadf['Blind'] = blind
    datadf['AlertTone'] = alerttone
    
    datadf = datadf[datadf['TimeDiff'] < 1000]
    
    if len(datadf) <= 0: return
    
    if plot:
        layout = go.Layout(
            title='Blind Error',
        )
        data = [
            go.Scatter(y=datadf[(datadf['AlertTone'] == True) & (datadf['Blind'] == True)]['error'],mode='line',name="Alert"),
            go.Scatter(y=datadf[(datadf['AlertTone'] == False) & (datadf['Blind'] == True)]['error'],mode='line',name="Non-Alert")
        ]
        fig = go.Figure(data=data, layout=layout)
        iplot(fig,filename=os.path.split(datafilename)[-1]+"-Error-Blind",image='png')

        layout = go.Layout(
                title='Non-blind Error',
        )

        data = [
            go.Scatter(y=datadf[(datadf['AlertTone'] == True) & (datadf['Blind'] == False)]['error'],mode='line',name="Alert"),
            go.Scatter(y=datadf[(datadf['AlertTone'] == False) & (datadf['Blind'] == False)]['error'],mode='line',name="Non-Alert")
        ]
        fig = go.Figure(data=data, layout=layout)
        iplot(fig,filename=os.path.split(datafilename)[-1]+"-Error-Non-Blind",image='png')
        
    fnamerow = [os.path.split(datafilename)[-1] for i in range(0,len(datadf))]
    
    datadf['Filename'] = fnamerow

    return datadf

## Basic Constants


In [3]:
radius = 0.85
targets = 9
targetsize = 0.45 # recheck

In [4]:
def get_target_pos(target_id):
    return (radius * math.cos(2.0 * math.pi * (target_id / 9.0)), radius * math.sin(2.0 * math.pi * (target_id / 9.0)))

def get_distance(target_id1, target_id2):
    pos1 = get_target_pos(int(target_id1[6:]))
    pos2 = get_target_pos(int(target_id2[6:]))
    return math.sqrt(math.pow(pos1[0]-pos2[0],2)+math.pow(pos1[1]-pos2[1],2))

def plot_fitts():
    x = []
    y = []
    labels = []
    for i in range(0,9):
        tmp = get_target_pos(i)
        x.append(tmp[0])
        y.append(tmp[1])
        labels.append(i)
    trace = go.Scatter(
        x = x,
        y = y,
        mode = 'markers',
        marker = dict(
            size = 65,
            color = 'rgba(152, 0, 0, .8)',
            line = dict(
                width = 2,
                color = 'rgb(0, 0, 0)'
            )
        ),
        text = labels
    )
    trace2 = go.Scatter(
        x = x,
        y = y,
        mode = 'lines'
    )
    data = [trace,trace2]
    layout = go.Layout(
        width = 500,
        height = 500,
        hovermode= 'closest',
        showlegend= False
    )
#     fig = go.Figure(data=data, layout=layout)
#     iplot(fig)
    return data,layout

In [5]:
def plot_csv(filename,plot=True):
    targetfilename = filename+".csv"
    datafilename = filename+".data.csv"
    targetdf = pd.read_csv(targetfilename)
    timingdata = dict()
    timingdata['time'] = []
    timingdata['session'] = []
    timingdata['round'] = []
    timingdata['trial'] = []
    timingdata['target_id'] = []
    timingdata['distance'] = []
    timingdata['blind'] = []
    timingdata['alerttone'] = []

    data,layout = plot_fitts()
    x = []
    y = []

    for i in range(1,len(targetdf)):
        cur = targetdf.iloc[i]
        if ((cur['Trial'] == -1) or (i == len(targetdf)-1)) and len(x) > 0:
            if len(x) > 0:
                trace = go.Scatter(x=x,y=y,mode="lines",line=dict(color='rgba(67,67,67,0.1)'))
                data.append(trace)
        if cur['Trial'] == -1: continue
        if targetdf.iloc[i-1]['TargetID'] == "-1": continue
        targetpos = get_target_pos(int(cur['TargetID'][6:]))
        x.append(targetpos[0])
        y.append(targetpos[1])
        timingdata['time'].append(cur['Millis']-targetdf.iloc[i-1]['Millis'])
        timingdata['session'].append(cur['Session'])
        timingdata['round'].append(cur['Round'])
        timingdata['trial'].append(cur['Trial'])
        timingdata['target_id'].append(cur['TargetID'])
        timingdata['blind'].append(cur['Blind'])
        timingdata['alerttone'].append(cur['AlertTone'])
        timingdata['distance'].append(get_distance(cur['TargetID'],targetdf.iloc[i-1]['TargetID']))

    layout = go.Layout(
        title = 'Selection density plot',
        width = 500,
        height = 500,
        hovermode= 'closest',
        showlegend= False
    )

    if(plot):
        fig = go.Figure(data=data, layout=layout)
        iplot(fig)
        py.image.save_as(fig, filename=targetfilename+"-Density.png")

    timingdf = pd.DataFrame.from_dict(timingdata)
    
    datablind=[]
    datasee=[]
    for i in range(1,5):
        sessiondf = timingdf[timingdf.session == i]
        if len(sessiondf) <= 0: continue
        trace = go.Scatter(
            x=range(0,len(sessiondf)),
            y=sessiondf['time'],
            mode='line',
            name=("No " if sessiondf.iloc[0]['alerttone']==False else "")+"Alert"
        )
        if sessiondf.iloc[0]['blind']:
            datablind.append(trace)
        else:
            datasee.append(trace)

    if len(datablind) > 0 and plot:
        layout = go.Layout(
            title='Blind Timings',
        )
        fig = go.Figure(data=datablind,layout=layout)
        iplot(fig,filename=os.path.split(targetfilename)[-1]+"-Blind",image='png')

    if len(datasee) > 0 and plot:
        layout = go.Layout(
            title='Non-blind Timings',
        )
        fig = go.Figure(data=datasee,layout=layout)
        iplot(fig,filename=os.path.split(targetfilename)[-1]+"-Non-Blind",image='png')
        
    fnamerow = [os.path.split(targetfilename)[-1] for i in range(0,len(timingdf))]
    
    timingdf['Filename'] = fnamerow

    return timingdf

## Overall statistics

In [36]:
csvfiles = glob("Data Readings/*[0-9].csv")
timingdfs = []
for csvfile in tqdm_notebook(csvfiles):
    timingdfs.append(plot_csv(csvfile[:-4],plot=False))
alltimingdfs = pd.concat(timingdfs)

In [29]:
datacsvfiles = glob("/Users/hrishioa/Desktop/Data Readings/*[0-9].data.csv")
datadfs = []
for datacsvfile in tqdm_notebook(datacsvfiles):
    datadfs.append(process_datadataframe(datacsvfile[:-9],plot=False))
alldatadfs = pd.concat(datadfs)

In [56]:
abdf = alltimingdfs[alltimingdfs.alerttone & 
                    alltimingdfs.blind & 
                    (alltimingdfs.time > 5)]
ab = np.array(abdf['time'],dtype=np.float64)

nabdf = alltimingdfs[(alltimingdfs.alerttone==False) & 
                     alltimingdfs.blind & 
                     (alltimingdfs.time > 2000) & 
                     (alltimingdfs.time > 5)]
nab = np.array(nabdf['time'],dtype=np.float64)

anbdf = alltimingdfs[alltimingdfs.alerttone & 
                     (alltimingdfs.blind == False) & 
                     (alltimingdfs.time < 20000) & 
                     (alltimingdfs.time > 5)]
anb = np.array(anbdf['time'],dtype=np.float64)

nanbdf = alltimingdfs[(alltimingdfs.alerttone==False) & 
                      (alltimingdfs.blind == False) & 
                      (alltimingdfs.time < 5500) & 
                      (alltimingdfs.time > 5)]
nanb = np.array(nanbdf['time'],dtype=np.float64)

ab = ab[:len(nab)]

trace1 = go.Bar(
    x=['Blind Mean', 'Non-Blind Mean', 'Blind Median', 'Non-Blind Median'],
    y=[
        np.mean(ab),np.mean(anb),
        np.percentile(ab,50),np.percentile(anb,50)
    ],
    name='Alert',
    error_y=dict(
        type='data',
        symmetric=False,
        array=[np.percentile(ab,75),np.percentile(anb,75),np.percentile(ab,75),np.percentile(anb,75)],
        arrayminus=[np.percentile(ab,25),np.percentile(anb,25),np.percentile(ab,25),np.percentile(anb,25)],
        visible=True
    )
)
trace2 = go.Bar(
    x=['Blind Mean', 'Non-Blind Mean', 'Blind Median', 'Non-Blind Median'],
    y=[
        np.mean(nab),np.mean(nanb),
        np.percentile(nab,50),np.percentile(nanb,50)
    ],
    name='Non-Alert',
    error_y=dict(
        type='data',
        symmetric=False,
        array=[np.percentile(nab,75),np.percentile(nanb,75),np.percentile(nab,75),np.percentile(nanb,75)],
        arrayminus=[np.percentile(nab,25),np.percentile(nanb,25),np.percentile(nab,25),np.percentile(nanb,25)],
        visible=True
    )
)
data = [trace1, trace2]
layout = go.Layout(
    title="Time to Target",
    barmode='group'
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename="timetotarget")

data = [
    go.Scatter(
            x=range(0,len(ab)),
            y=ab,
            mode='line',
            name="Alert"
        ),
    go.Scatter(
        x=range(0,len(nab)),
        y=nab,
        mode='line',
        name="Non-Alert"
    )
]
layout = go.Layout(
    title='Blind Timings',
)
fig = go.Figure(data=data,layout=layout)
iplot(fig,filename="blindtimings")



data = [
    go.Scatter(
            x=range(0,len(anb)),
            y=anb,
            mode='line',
            name="Alert"
        ),
    go.Scatter(
        x=range(0,len(nanb)),
        y=nanb,
        mode='line',
        name="Non-Alert"
    )
]
layout = go.Layout(
    title='Non-Blind Timings',
)
fig = go.Figure(data=data,layout=layout)
iplot(fig,filename="nonblindtimings")


In [10]:
abdf = alltimingdfs[alltimingdfs.alerttone & 
                    alltimingdfs.blind & 
                    (alltimingdfs.time > 5)]
ab = np.array(abdf['time'],dtype=np.float64)/np.array(abdf['distance'])

nabdf = alltimingdfs[(alltimingdfs.alerttone==False) & 
                     alltimingdfs.blind & 
                     (alltimingdfs.time > 2000) & 
                     (alltimingdfs.time > 5)]
nab = np.array(nabdf['time'],dtype=np.float64)/np.array(nabdf['distance'])

anbdf = alltimingdfs[alltimingdfs.alerttone & 
                     (alltimingdfs.blind == False) & 
                     (alltimingdfs.time < 20000) & 
                     (alltimingdfs.time > 5)]
anb = np.array(anbdf['time'],dtype=np.float64)/np.array(anbdf['distance'])

nanbdf = alltimingdfs[(alltimingdfs.alerttone==False) & 
                      (alltimingdfs.blind == False) & 
                      (alltimingdfs.time < 20000) & 
                      (alltimingdfs.time > 5)]
nanb = np.array(nanbdf['time'],dtype=np.float64)/np.array(nanbdf['distance'])

ab = ab[:len(nab)]

blind = [(True if i < len(ab)+len(nab) else False) for i in range(0,len(ab)+len(nab)+len(anb)+len(nanb))]
alert = [True for i in range(0,len(ab))]+[False for i in range(0,len(nab))]+[True for i in range(0,len(anb))]+[False for i in range(0,len(nanb))]
error_df = pd.DataFrame({'time':np.concatenate((ab,nab,anb,nanb)),'blind':blind,'alert':alert})

trace1 = go.Bar(
    x=['Blind Mean', 'Non-Blind Mean', 'Blind Median', 'Non-Blind Median'],
    y=[
        np.mean(ab),np.mean(anb),
        np.percentile(ab,50),np.percentile(anb,50)
    ],
    name='Alert',
    error_y=dict(
        type='data',
        symmetric=False,
        array=[np.percentile(ab,75),np.percentile(anb,75),np.percentile(ab,75),np.percentile(anb,75)],
        arrayminus=[np.percentile(ab,25),np.percentile(anb,25),np.percentile(ab,25),np.percentile(anb,25)],
        visible=True
    )
)
trace2 = go.Bar(
    x=['Blind Mean', 'Non-Blind Mean', 'Blind Median', 'Non-Blind Median'],
    y=[
        np.mean(nab),np.mean(nanb),
        np.percentile(nab,50),np.percentile(nanb,50)
    ],
    name='Non-Alert',
    error_y=dict(
        type='data',
        symmetric=False,
        array=[np.percentile(nab,75),np.percentile(nanb,75),np.percentile(nab,75),np.percentile(nanb,75)],
        arrayminus=[np.percentile(nab,25),np.percentile(nanb,25),np.percentile(nab,25),np.percentile(nanb,25)],
        visible=True
    )
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group',
    title="Time to Target Over Distance"
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='timeoverdistancetotarget')


data = [
    go.Scatter(
            x=range(0,len(ab)),
            y=(ab),
            mode='line',
            name="Alert"
        ),
    go.Scatter(
        x=range(0,len(nab)),
        y=(nab),
        mode='line',
        name="Non-Alert"
    )
]
layout = go.Layout(
    title='Blind Timings over distance',
)
fig = go.Figure(data=data,layout=layout)
iplot(fig,filename="blindtimingdistance")

data = [
    go.Scatter(
            x=range(0,len(anb)),
            y=anb,
            mode='line',
            name="Alert"
        ),
    go.Scatter(
        x=range(0,len(nanb)),
        y=nanb,
        mode='line',
        name="Non-Alert"
    )
]
layout = go.Layout(
    title='Non-Blind Timings over distance',
)
fig = go.Figure(data=data,layout=layout)
iplot(fig,filename="nonblindtimingdistance")

In [58]:
abdf = alltimingdfs[alltimingdfs.alerttone & 
                    alltimingdfs.blind & 
                    (alltimingdfs.time > 5)]
ab = np.array(abdf['time'],dtype=np.float64)/np.array(abdf['distance'])

nabdf = alltimingdfs[(alltimingdfs.alerttone==False) & 
                     alltimingdfs.blind & 
                     (alltimingdfs.time > 2000) & 
                     (alltimingdfs.time > 5)]
nab = np.array(nabdf['time'],dtype=np.float64)/np.array(nabdf['distance'])

anbdf = alltimingdfs[alltimingdfs.alerttone & 
                     (alltimingdfs.blind == False) & 
                     (alltimingdfs.time < 20000) & 
                     (alltimingdfs.time > 5)]
anb = np.array(anbdf['time'],dtype=np.float64)/np.array(anbdf['distance'])

nanbdf = alltimingdfs[(alltimingdfs.alerttone==False) & 
                      (alltimingdfs.blind == False) & 
                      (alltimingdfs.time < 20000) & 
                      (alltimingdfs.time > 5)]
nanb = np.array(nanbdf['time'],dtype=np.float64)/np.array(nanbdf['distance'])

ab = ab[:len(nab)]

blind = [(True if i < len(ab)+len(nab) else False) for i in range(0,len(ab)+len(nab)+len(anb)+len(nanb))]
alert = [True for i in range(0,len(ab))]+[False for i in range(0,len(nab))]+[True for i in range(0,len(anb))]+[False for i in range(0,len(nanb))]
error_df = pd.DataFrame({'time':np.concatenate((ab,nab,anb,nanb)),'blind':blind,'alert':alert})

trace1 = go.Bar(
    x=['Blind', 'Non-Blind'],
    y=[
        np.mean(ab),np.mean(anb)
    ],
    name='Alert',
    error_y=dict(
        type='data',
        symmetric=False,
        array=[np.percentile(ab,75),np.percentile(anb,75)],
        arrayminus=[np.percentile(ab,25),np.percentile(anb,25)],
        visible=True
    )
)
trace2 = go.Bar(
    x=['Blind', 'Non-Blind'],
    y=[
        np.mean(nab),np.mean(nanb)
    ],
    name='Non-Alert',
    error_y=dict(
        type='data',
        symmetric=False,
        array=[np.percentile(nab,75),np.percentile(nanb,75)],
        arrayminus=[np.percentile(nab,25),np.percentile(nanb,25)],
        visible=True
    )
)

data = [trace1, trace2]
layout = go.Layout(
    width=700,
    height=600,
    barmode='group',
    title="Average Time to Target"
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='timetotarget')


data = [
    go.Scatter(
            x=range(0,len(ab)),
            y=(ab),
            mode='line',
            name="Alert"
        ),
    go.Scatter(
        x=range(0,len(nab)),
        y=(nab),
        mode='line',
        name="Non-Alert"
    )
]
layout = go.Layout(
    width=450,
    height=300,
    title='Blind Time to target',
)
fig = go.Figure(data=data,layout=layout)
py.iplot(fig,filename="blindtiming",image='png')

data = [
    go.Scatter(
            x=range(0,len(anb)),
            y=anb,
            mode='line',
            name="Alert"
        ),
    go.Scatter(
        x=range(0,len(nanb)),
        y=nanb,
        mode='line',
        name="Non-Alert"
    )
]
layout = go.Layout(
    width=450,
    font=dict(size=14),
    height=300,
    title='Non-Blind Time to target',
)
fig = go.Figure(data=data,layout=layout)
py.iplot(fig,filename="nonblindtiming",image='png')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~toonistic/0 or inside your plot.ly account where it is named 'timetotarget'


In [49]:
from scipy import stats

slope, intercept, r_value, p_value, std_err = stats.linregress(nabdf['distance'],nabdf['time'])
line = slope*nabdf['distance']+intercept

slope, intercept, r_value, p_value, std_err = stats.linregress(abdf['distance'],abdf['time'])
line2 = slope*abdf['distance']+intercept

data = [
    go.Scatter(
            x=nabdf['distance'],
            y=nabdf['time'],
            mode='markers',
            name="NAB"
        ),
    go.Scatter(
            x=nabdf['distance'],
            y=line,
            mode='lines',
            name="NABLINE"
        ),
    go.Scatter(
            x=abdf['distance'],
            y=abdf['time'],
            mode='markers',
            name="AB"
        ),
    go.Scatter(
            x=abdf['distance'],
            y=line2,
            mode='lines',
            name="ABLINE"
        ),
]
layout = go.Layout(
    title='Blind Alert Time against Distance',
)
fig = go.Figure(data=data,layout=layout)
iplot(fig,filename="blindalerttimedistancebestfit")

## Distance to Target

In [50]:
factor=1.0
ab = abs(np.array(alldatadfs[alldatadfs.AlertTone==False & alldatadfs.Blind ]['error']))*factor
anb = abs(np.array(alldatadfs[alldatadfs.AlertTone==False & (alldatadfs.Blind == False)]['error']))*factor
nab = abs(np.array(alldatadfs[(alldatadfs.AlertTone) & alldatadfs.Blind]['error']))*factor
nanb = abs(np.array(alldatadfs[(alldatadfs.AlertTone) & (alldatadfs.Blind == False)]['error']))*factor

nab = nab[:len(ab)]

blind = [(True if i < len(ab)+len(nab) else False) for i in range(0,len(ab)+len(nab)+len(anb)+len(nanb))]
alert = [True for i in range(0,len(ab))]+[False for i in range(0,len(nab))]+[True for i in range(0,len(anb))]+[False for i in range(0,len(nanb))]
timedistance_df = pd.DataFrame({'time':np.concatenate((ab,nab,anb,nanb)),'blind':blind,'alert':alert})

trace1 = go.Bar(
    x=['Blind', 'Non-Blind'],
    y=[
        np.mean(ab),np.mean(anb)
    ],
    name='Alert',
    error_y=dict(
        type='data',
        symmetric=False,
        array=[np.percentile(ab,75),np.percentile(anb,75)],
        arrayminus=[np.percentile(ab,25),np.percentile(anb,25)],
        visible=True
    )
)
trace2 = go.Bar(
    x=['Blind', 'Non-Blind'],
    y=[
        np.mean(nab),np.mean(nanb),
    ],
    name='Non-Alert',
    error_y=dict(
        type='data',
        symmetric=False,
        array=[np.percentile(nab,75),np.percentile(nanb,75)],
        arrayminus=[np.percentile(nab,25),np.percentile(nanb,25)],
        visible=True
    )
)

data = [trace1, trace2]
layout = go.Layout(
    width=700,
    height=600,
    title="Target Error",
    barmode='group'
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='distancetotarget',image='png')


data = [
    go.Scatter(
            x=range(0,len(ab)),
            y=ab,
            mode='line',
            name="Alert"
        ),
    go.Scatter(
        x=range(0,len(nab)),
        y=nab,
        mode='line',
        name="Non-Alert"
    )
]
layout = go.Layout(
    title='Blind Distance',
)
fig = go.Figure(data=data,layout=layout)
iplot(fig,filename="blinddistance")


data = [
    go.Scatter(
            x=range(0,len(anb)),
            y=anb,
            mode='line',
            name="Alert"
        ),
    go.Scatter(
        x=range(0,len(nanb)),
        y=nanb,
        mode='line',
        name="Non-Alert"
    )
]
layout = go.Layout(
    title='Non-Blind Distance',
)
fig = go.Figure(data=data,layout=layout)
iplot(fig,filename="nonblinddistance")

## ANOVA

In [27]:
import scipy

import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
from plotly.tools import FigureFactory as FF

/Users/hrishioa/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



In [51]:
formula = 'error ~ C(AlertTone) + C(Blind) + C(AlertTone):C(Blind)'
model = ols(formula, alldatadfs).fit()
aov_table = statsmodels.stats.anova.anova_lm(model, typ=2)
print(aov_table)

                         sum_sq      df          F    PR(>F)
C(AlertTone)           0.000689     1.0   0.116334  0.733105
C(Blind)               0.105928     1.0  17.875135  0.000025
C(AlertTone):C(Blind)  0.029434     1.0   4.966876  0.026021
Residual               7.152667  1207.0        NaN       NaN


In [52]:
formula = 'timedistance ~ C(alerttone) + C(blind) + C(alerttone):C(blind)'
alltimingdfs['timedistance'] = alltimingdfs.time/alltimingdfs.distance
model = ols(formula, alltimingdfs).fit()
aov_table = statsmodels.stats.anova.anova_lm(model, typ=2)
print(aov_table)

                             sum_sq      df           F        PR(>F)
C(alerttone)           1.003286e+08     1.0    1.033471  3.095424e-01
C(blind)               4.621135e+10     1.0  476.016779  1.286587e-89
C(alerttone):C(blind)  2.551348e+08     1.0    2.628109  1.052401e-01
Residual               1.207666e+11  1244.0         NaN           NaN


## Phase 1

In [17]:
averagesize = 10000

blind1 = pd.read_csv("../tactileracer/Data/blind.csv")

bind1 = blind1.iloc[:34500]

currentsq = np.power(blind1.current,2)

averages = [np.mean(currentsq[i:i+averagesize]) for i in tqdm_notebook(range(0,len(blind1)-averagesize))]

blind2 = pd.read_csv("../tactileracer/Data/blind2.csv")

currentsq2 = np.power(blind2.current,2)

averages2 = [np.mean(currentsq2.iloc[i:i+averagesize]) for i in tqdm_notebook(range(0,len(blind2)-averagesize))]

vision1 = pd.read_csv("../tactileracer/Data/vision.csv")

currentsq3 = np.power(vision1.current,2)

averages3 = [np.mean(currentsq3[i:i+averagesize]) for i in tqdm_notebook(range(0,len(vision1)-averagesize))]

vision2 = pd.read_csv("../tactileracer/Data/vision2.csv")

currentsq4 = np.power(vision2.current,2)

averages4 = [np.mean(currentsq4.iloc[i:i+averagesize]) for i in tqdm_notebook(range(0,len(vision2)-averagesize))]

data = [
    go.Scatter(
            y=averages2,
            mode='line',
            name="Blind"
        ),
    go.Scatter(
            y=averages4,
            mode='line',
            name="Non-Blind"
        ),
    go.Scatter(
            y=np.array(averages4)/np.array(averages2[:len(averages4)]),
            mode='line',
            name="Blind/Non-Blind"
        )
]
layout = go.Layout(
    width=700,
    height=400,
    title='Phase 1 Error against Time',
)
fig = go.Figure(data=data,layout=layout)
py.iplot(fig,filename="p1error")

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.



The draw time for this plot will be slow for clients without much RAM.


/Users/hrishioa/anaconda/lib/python2.7/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow



## Deeper Analysis

In [46]:
csvfiles = glob("Data Readings/*[0-9].csv")
timingdfs = []
for csvfile in tqdm_notebook(csvfiles):
    timingdfs.append(plot_csv(csvfile[:-4],plot=False))
alltimingdfs = pd.concat(timingdfs)
print [np.mean(curdf[curdf.alerttone & curdf.blind]['time']) for curdf in timingdfs]
print [np.mean(curdf[curdf.alerttone & (curdf.blind==False)]['time']) for curdf in timingdfs]
print [np.mean(curdf[(curdf.alerttone==False) & curdf.blind]['time']) for curdf in timingdfs]
print [np.mean(curdf[(curdf.alerttone==False) & (curdf.blind==False)]['time']) for curdf in timingdfs]
csvfiles


[10797.825, 10143.05, 7249.825, 20129.875, 13042.475, 31838.075, 29749.25, 7835.325]
[703.075, 766.95, 852.525, 1230.575, 1109.825, 640.1, 641.675, 1221.2]
[7044.025, 7929.125, 8855.075, 24447.625, 13885.35, 20182.225, 21036.05, 11806.025]
[907.1, 741.025, 846.025, 1045.65, 831.375, 680.4, 710.85, 4312.95]


['Data Readings/Jay26032018T191933.csv',
 'Data Readings/Abhijath26032018T205340.csv',
 'Data Readings/Abdullah26032018T193833.csv',
 'Data Readings/Aaron14032018T020257.csv',
 'Data Readings/Hebe26032018T181631.csv',
 'Data Readings/Parag113032018T062226.csv',
 'Data Readings/Somaletha14032018T013807.csv',
 'Data Readings/Hansel26032018T190038.csv']

In [55]:
nonalert_nonblind = go.Bar(
    x=range(0,len(timingdfs)),
    y=[np.mean(curdf[(curdf.alerttone==False) & (curdf.blind==False)]['time']) for curdf in timingdfs],
    name='Non-Alert Non-Blind',
    error_y=dict(
        type='data',
        symmetric=False,
        array=[np.percentile(curdf[(curdf.alerttone==False) & (curdf.blind==False)]['time'],75) for curdf in timingdfs],
        arrayminus=[np.percentile(curdf[(curdf.alerttone==False) & (curdf.blind==False)]['time'],25) for curdf in timingdfs],
        visible=True
    )
)

nonalert_blind = go.Bar(
    x=range(0,len(timingdfs)),
    y=[np.mean(curdf[(curdf.alerttone==False) & curdf.blind]['time']) for curdf in timingdfs],
    name='Non-Alert Blind',
    error_y=dict(
        type='data',
        symmetric=False,
        array=[np.percentile(curdf[(curdf.alerttone==False) & curdf.blind]['time'],75) for curdf in timingdfs],
        arrayminus=[np.percentile(curdf[(curdf.alerttone==False) & curdf.blind]['time'],25) for curdf in timingdfs],
        visible=True
    )
)

alert_nonblind = go.Bar(
    x=range(0,len(timingdfs)),
    y=[np.mean(curdf[curdf.alerttone & (curdf.blind==False)]['time']) for curdf in timingdfs],
    name='Alert Non-Blind',
    error_y=dict(
        type='data',
        symmetric=False,
        array=[np.percentile(curdf[curdf.alerttone & (curdf.blind==False)]['time'],75) for curdf in timingdfs],
        arrayminus=[np.percentile(curdf[curdf.alerttone & (curdf.blind==False)]['time'],25) for curdf in timingdfs],
        visible=True
    )
)

alert_blind = go.Bar(
    x=range(0,len(timingdfs)),
    y=[np.mean(curdf[curdf.alerttone & curdf.blind]['time']) for curdf in timingdfs],
    name='Alert Blind',
    error_y=dict(
        type='data',
        symmetric=False,
        array=[np.percentile(curdf[curdf.alerttone & curdf.blind]['time'],75) for curdf in timingdfs],
        arrayminus=[np.percentile(curdf[curdf.alerttone & curdf.blind]['time'],25) for curdf in timingdfs],
        visible=True
    )
)

data = [alert_blind, nonalert_blind, alert_nonblind, nonalert_nonblind]
layout = go.Layout(
    title="Characteristics against Individuals",
    font=dict(size=16),
    barmode='group'
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename="timetotarget")
